In [1]:
from rag import get_docs, split_text, get_data_store, predict, postprocess_response

/home/pauli/.cache/pypoetry/virtualenvs/image-lyrics-hleqkuGI-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
config = {
    "llm": {
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.2,
        "repetition_penalty": 1.2
    },
    "retriever": {
        "search_type": "mmr",
        "k": 10
    },
    "splitter": {
        "chunk_size": 250,
        "chunk_overlap": 25
    }
}

In [3]:
IMAGES = [
    "mt_fuji.jpg",
    "holland.jpg",
    "new_york.jpg",
    "videoconference.jpg"
]

In [4]:
docs = get_docs()

In [5]:
chunks = split_text(docs, config) 
db = get_data_store(chunks)

/home/pauli/Git/image-lyrics/rag.py:61: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings( #  embedding=OpenAIEmbeddings() rate limit
/home/pauli/.cache/pypoetry/virtualenvs/image-lyrics-hleqkuGI-py3.10/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
answer, retrieved_chunks, prompt, _ = predict(IMAGES, db, config)

/home/pauli/.cache/pypoetry/virtualenvs/image-lyrics-hleqkuGI-py3.10/lib/python3.10/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [17]:
print(prompt)

Write a song lyrics about "mountain in the distance with a lake and red leavesa close up of a windmill in a field of flowerscity street with cars and pedestrians in the middle of the streetwoman sitting at a desk with a laptop and a notebook" having intro, verse 1, chorus 1, verse 2, chorus 2, bridge, chorus 3, outro. Ignore the words intro, verse, chorus and outro.


In [7]:
import nltk

In [8]:
from bert_score import BERTScorer
from bleu_score import calculate_blue_score

In [10]:
"""
scorer = BERTScorer(model_type='bert-base-uncased')
for i in range(len(retrieved_chunks)):
    sentences = [sentence for sentence in answer.split('\n') if sentence != ""]
    print(retrieved_chunks[i])
    print('\n')
    for sentence in sentences:
        BLEUscore, _, _ = scorer.score([sentence], [retrieved_chunks[i]])
        print(BLEUscore.item(), sentence)
    print('\n')
"""

'\nscorer = BERTScorer(model_type=\'bert-base-uncased\')\nfor i in range(len(retrieved_chunks)):\n    sentences = [sentence for sentence in answer.split(\'\n\') if sentence != ""]\n    print(retrieved_chunks[i])\n    print(\'\n\')\n    for sentence in sentences:\n        BLEUscore, _, _ = scorer.score([sentence], [retrieved_chunks[i]])\n        print(BLEUscore.item(), sentence)\n    print(\'\n\')\n'

In [15]:
from langchain.embeddings import HuggingFaceEmbeddings
from numpy import dot
from numpy.linalg import norm

embeddings = HuggingFaceEmbeddings( 
    model_name='sentence-transformers/all-MiniLM-L6-v2',
    model_kwargs={'device': 'cpu'} 
)

# Get the embedding
for i in range(len(retrieved_chunks)):
    sentences = [sentence for sentence in answer.split('\n') if sentence != ""]
    print(retrieved_chunks[i])
    print('\n')
    chunk_sentences = [sentence for sentence in retrieved_chunks[i].split('\n') if sentence != ""]
    for chunk_sentence in chunk_sentences:
        print(chunk_sentence)
        print('\n')
        for sentence in sentences:
            embedding_vector_sentence = embeddings.embed_query(sentence) 
            embeddings_vector_chunk = embeddings.embed_query(chunk_sentence)
            cos_sim = dot(embedding_vector_sentence, embeddings_vector_chunk)/(norm(embedding_vector_sentence)*norm(embeddings_vector_chunk))
            if cos_sim > 0.4:
                print(cos_sim, sentence)
        print('\n')

/home/pauli/.cache/pypoetry/virtualenvs/image-lyrics-hleqkuGI-py3.10/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[Intro]
Please, please tell me now
Please, please tell me now
Please, please tell me now
Please, please tell me now
[Verse 1]
I made a break, I run out yesterday
Tried to find my mountain hideaway
Maybe next year, maybe no go


[Intro]


0.801115731486393 Intro:


Please, please tell me now




Please, please tell me now




Please, please tell me now




Please, please tell me now




[Verse 1]


0.8725954142430002 Verse 1:
0.4644465375686951 Chorus 1:
0.7938342240062902 Verse 2:
0.42035978318878164 Chorus 2:
0.40695460492939917 Chorus 3:


I made a break, I run out yesterday




Tried to find my mountain hideaway


0.4132427530941906 A mountain stands tall in the distance,
0.418199830222423 Mountains rise, lakes glisten,


Maybe next year, maybe no go




[Bridge]
(Hoo hoo hoo)
[Verse 2]Reach your hand down into the cooler
Don't drink it if the mountains aren't blueTry to keep it steady as you recline on your black inner tube
[Chorus]
[Bridge]
(Hoo hoo hoo)
(Hoo hoo hoo)


[Bridge]



In [29]:
from langchain.embeddings import HuggingFaceEmbeddings
from numpy import dot
from numpy.linalg import norm

embeddings = HuggingFaceEmbeddings( 
    model_name='sentence-transformers/all-MiniLM-L6-v2',
    model_kwargs={'device': 'cpu'} 
)
true_claims = 0
all_claims = 0
# Get the embedding
sentences = [sentence for sentence in answer.split('\n') if sentence != ""]
claims = [0] * len(sentences)

for i in range(len(retrieved_chunks)):
    
    print(retrieved_chunks[i])
    print('\n')
    chunk_sentences = [sentence for sentence in retrieved_chunks[i].split('\n') if sentence != ""]
    for chunk_sentence in chunk_sentences:
        print(chunk_sentence)
        print('\n')
        for i in range(len(sentences)):
            embedding_vector_sentence = embeddings.embed_query(sentences[i]) 
            embeddings_vector_chunk = embeddings.embed_query(chunk_sentence)
            cos_sim = dot(embedding_vector_sentence, embeddings_vector_chunk)/(norm(embedding_vector_sentence)*norm(embeddings_vector_chunk))
            if cos_sim > 0.4:
                print(1, sentences[i])
                claims[i] = 1
            else:
                print(0, sentences[i])
        print('\n')

print("true claims: ", sum(claims))
print("all claims: ", len(sentences))
faithfulness = sum(claims) / len(sentences)
print("faithfulness: ", faithfulness)

/home/pauli/.cache/pypoetry/virtualenvs/image-lyrics-hleqkuGI-py3.10/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[Intro]
Please, please tell me now
Please, please tell me now
Please, please tell me now
Please, please tell me now
[Verse 1]
I made a break, I run out yesterday
Tried to find my mountain hideaway
Maybe next year, maybe no go


[Intro]


1 Intro:
0 As the sun begins to set,
0 A peaceful scene unfolds,
0 A mountain stands tall in the distance,
0 Reflecting orange and gold.
0 Below its majestic form,
0 A tranquil lake shimmers,
0 Red leaves dance upon its surface,
0 Nature's symphony lingers.
0 Close Up:
0 Amidst a vast expanse of green,
0 A windmill spins in harmony,
0 Petals sway in gentle breeze,
0 Flowering fields are free.
0 City Street:
0 Cars honk, people hurry past,
0 Pedestrians dodge their way,
0 Melodies blend with urban noise,
0 Life moves forward each day.
0 Woman Sitting:
0 She sits at her desk, fingers tap,
0 Notebook open wide,
0 Her thoughts flow freely,
0 Creativity comes alive.
0 Verse 1:
0 Watch the colors fade,
0 As darkness creeps in sight,
0 Stillness reigns across

In [30]:
for i in range(len(sentences)):
    if claims[i] == 1:
        print(1, sentences[i])
    else:
        print(0, sentences[i])
print('\n')

1 Intro:
1 As the sun begins to set,
0 A peaceful scene unfolds,
1 A mountain stands tall in the distance,
1 Reflecting orange and gold.
1 Below its majestic form,
1 A tranquil lake shimmers,
0 Red leaves dance upon its surface,
0 Nature's symphony lingers.
0 Close Up:
1 Amidst a vast expanse of green,
1 A windmill spins in harmony,
0 Petals sway in gentle breeze,
0 Flowering fields are free.
1 City Street:
0 Cars honk, people hurry past,
1 Pedestrians dodge their way,
1 Melodies blend with urban noise,
0 Life moves forward each day.
0 Woman Sitting:
0 She sits at her desk, fingers tap,
0 Notebook open wide,
0 Her thoughts flow freely,
0 Creativity comes alive.
1 Verse 1:
0 Watch the colors fade,
0 As darkness creeps in sight,
0 Stillness reigns across the landscape,
0 Peace descends, all is right.
1 Chorus 1:
1 Mountains rise, lakes glisten,
0 Leaves ablaze in fiery hues,
1 Windmills spin, flowers bloom,
1 City life bustles anew.
1 Verse 2:
0 Days pass, seasons change,
0 Yet beauty re

In [12]:
print(answer)

Intro:
In the distance, a mountain stands tall
Its peak covered in red leaves that fall
A lake shimmers below, reflecting its hue
Nature's beauty, so pure and true

Verse 1:
The windmill spins, blades turning round
Golden fields of flowers all around
City streets bustle with life
Cars honk, pedestrians cross the strife

Chorus 1:
Mountains in the distance, lakes aglow
Windmills spinning, nature's show
City streets alive with sound
Beauty surrounds us, let it astound

Verse 2:
At her desk, she types away
Notebook open, ideas at play
Her laptop glows, words on display
Creativity flows, day by day

Chorus 2:
Red leaves falling, mountains still high
Lake sparkling, sky up above we'll fly
City lights twinkling, nightlife ablaze
Life moves forward, never to be replaced

Bridge:
(Hoo Hoo Hoo)

Chorus 3:
Nature's splendor, city's beat
All around us, our hearts skip a beat
Let's cherish this moment, forevermore
For life is fleeting, but memories endure

Outro:
As the sun sets, colors fade
Leave

In [31]:
from metrics import calculate_faithfulness

In [32]:
calculate_faithfulness(answer, retrieved_chunks)

/home/pauli/.cache/pypoetry/virtualenvs/image-lyrics-hleqkuGI-py3.10/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


0.4067796610169492